# DNANexus Login

In [1]:
!pip install dxpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.3/602.3 kB 7.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Using cached websocket_client-0.54.0-py2.py3-none-any.whl (200 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 14.9 MB/s eta 0:00:00
  Using cached urllib3-2.1.0-py3-none-any.whl (104 kB)
  Using cached argcomplete-1.12.3-py2.py3-none-any.whl (38 kB)
  DEPRECATION: dxpy is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for dxpy ... done

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import dxpy as dx

DX_SECURITY_CONTEXT = {
        "auth_token_type": "Bearer",
        "auth_token": ''
    }

dx.set_security_context(DX_SECURITY_CONTEXT)

In [3]:
dx.whoami()

'user-jason_l'

# Load RNAS

In [4]:
with open('rnas.txt', 'r', encoding='utf-8-sig') as f:
  rnas = set([v for v in f.read().splitlines() if v])

In [5]:
print("number of RNAs:", len(rnas))

number of RNAs: 45


# GenePanel Comparison

In [29]:
# read genepanel file from dnanexus

old_dxfile = dx.DXFile('file-Gj7ygzj42X4ZBqg9068p1fQ4') # 240405
new_dxfile = dx.DXFile('file-Gkjk6zQ433GyXvqbYGpFBFgx') # 240610

old_genepanel_dxfile = old_dxfile.read()
new_genepanel_dxfile = new_dxfile.read() # there is a newer genepanel file project-Fkb6Gkj433GVVvj73J7x8KbV:file-GVx0vkQ433Gvq63k1Kj4Y562

In [32]:
print(f'You are comparing {old_dxfile.name} with {new_dxfile.name}')

You are comparing 230602_genepanels.tsv with 240213_genepanels.tsv


In [33]:
import pandas as pd

# format tsv file into a df
old_genepanel_df = pd.DataFrame([line.split('\t') for line in old_genepanel_dxfile.split('\n')], columns=['Gemini Name', 'Panel', 'Gene'])
new_genepanel_df = pd.DataFrame([line.split('\t') for line in new_genepanel_dxfile.split('\n')], columns=['Gemini Name', 'Panel','Gene', "Panel ID"])

In [34]:
old_genepanel_df.dropna(inplace=True)
new_genepanel_df.dropna(inplace=True)

In [35]:
# Missing R codes in old and new genepanels (Addition and Deletion)

missing_r_codes = []

# get the r code without, regardless of version
new_r_codes = set([r.split('_')[0].strip().split('.')[0].strip() for r in new_genepanel_df['Gemini Name'].values.tolist()])
old_r_codes = set([r.split('_')[0].strip().split('.')[0].strip() for r in old_genepanel_df['Gemini Name'].values.tolist()])

missing_in_new_genepanels = list(sorted(old_r_codes - new_r_codes))
added_in_new_genepanels = list(sorted(new_r_codes - old_r_codes))

In [36]:
# doesn't take into account of value after the decimal R31.4

print(f'R Codes missing in new genepanel: {missing_in_new_genepanels}')
print(f'New R Codes added in new genepanel: {added_in_new_genepanels}')

R Codes missing in new genepanel: ['R434']
New R Codes added in new genepanel: ['R227', 'R391', 'R421', 'R82']


# checking missing or adding genes

In [37]:
import collections

old_r_code_to_ci_name = {}
old_r_code_to_panel = {}
old_r_code_to_genes = collections.defaultdict(list)


new_r_code_to_ci_name = {}
new_r_code_to_panel = {}
new_r_code_to_genes = collections.defaultdict(list)

print('processing old genepanel...')

for idx, row in old_genepanel_df.iterrows():
  gemini_name = row['Gemini Name'].strip()
  panel = row['Panel'].strip()
  gene = row['Gene'].strip()

  r_code, ci = gemini_name.split('_', maxsplit=1)

  if '_' in ci:
    ci = ci.split('_')[0]

  if r_code in new_r_code_to_panel and (
      (new_r_code_to_panel[r_code].startswith('HGNC') and not panel.startswith('HGNC')) or (not new_r_code_to_panel[r_code].startswith('HGNC') and panel.startswith('HGNC'))
      ):
    print(f'❌ {r_code} {ci} have two different kind of panels:\n-{new_r_code_to_panel[r_code]}\n-{panel}')

  old_r_code_to_ci_name[r_code] = ci
  old_r_code_to_panel[r_code] = panel
  old_r_code_to_genes[r_code].append(gene)

print('processing new genepanel...')

for idx, row in new_genepanel_df.iterrows():
  gemini_name = row['Gemini Name'].strip()
  panel = row['Panel'].strip()
  gene = row['Gene'].strip()

  if gemini_name.count('_') == 1:
    r_code, ci = gemini_name.split('_')
  else:
    r_code, ci, _ = gemini_name.split('_')

  if r_code in new_r_code_to_panel and (
      (new_r_code_to_panel[r_code].startswith('HGNC') and not panel.startswith('HGNC')) or (not new_r_code_to_panel[r_code].startswith('HGNC') and panel.startswith('HGNC'))
      ):
    print(f'❌ {r_code} {ci} have two different kind of panels:\n-{new_r_code_to_panel[r_code]}\n-{panel}')

  # this is manual intervention step. Remove to see the error purpose
  if r_code in ['R211.1', 'R293.1', 'R38.2'] and panel.startswith('HGNC'):
    continue

  new_r_code_to_ci_name[r_code] = ci
  new_r_code_to_panel[r_code] = panel
  new_r_code_to_genes[r_code].append(gene)

print('done.')

processing old genepanel...
processing new genepanel...
done.


In [38]:
!pip install fuzzywuzzy # for panel name matching


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [39]:
!pip install odfpy # for excel reading


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [40]:
old_td = pd.read_excel('test directories/Rare-and-inherited-disease-national-genomic-test-directory-version-5.1.xlsx', sheet_name=1, header=1)
new_td = pd.read_excel('test directories/Rare and Inherited Disease National Genomic Test Directory (GLH FACING) V6 TRACKED OCT 2023_modified.xlsx', sheet_name=1, header=1)

# included method might change depending on which version of test directory
new_included_test_methods = [
        "Medium panel", "Single gene sequencing <=10 amplicons",
        "Single gene sequencing <10 amplicons",
        "Single gene sequencing >=10 amplicons",
        "Single gene testing (<10 amplicons)", "small panel", "Small panel",
        "WES or Large panel", "WES or Large Panel", "WES or Medium Panel",
        "WES or Large penel", "WES or Medium panel", "WES or Small Panel",
        "WGS", "WES", "Exon level CNV detection by MLPA or equivalent"
    ]

old_included_test_methods = [
        "Medium panel", "Single gene sequencing <=10 amplicons",
        "Single gene sequencing <10 amplicons",
        "Single gene sequencing >=10 amplicons",
        "Single gene testing (<10 amplicons)", "small panel", "Small panel",
        "WES or Large panel", "WES or Large Panel", "WES or Medium Panel",
        "WES or Large penel", "WES or Medium panel", "WES or Small Panel",
        "WGS", "WES"
    ]

In [41]:
new_td = new_td[new_td['Test Method'].str.strip().str.lower().isin([m.lower() for m in new_included_test_methods])]
old_td = old_td[old_td['Test Method'].str.strip().str.lower().isin([m.lower() for m in old_included_test_methods])]

In [42]:
import requests

def get_genes_from_panelapp_id(panel_id, panel_version):
  """
  function to fetch genes from panelapp api given panel id and version
  """
  response = requests.get(f'https://panelapp.genomicsengland.co.uk/api/v1/panels/{panel_id}/?version={panel_version}')

  if response.status_code != 200:
    return None

  gene_data = [g['gene_data'] for g in response.json()['genes'] if g['confidence_level'] == '3']

  result = set()

  for gene in gene_data:
    if gene.get('hgnc_id'):
      result.add(gene['hgnc_id'])
    else:
      print(f'No hgnc id found for {gene_data}')

  return result

def query_gene_symbol(gene_symbol):
  """
  query gene symbol and return HGNC
  """

  response = requests.get(f"https://www.genenames.org/cgi-bin/search/search?query={gene_symbol}&rows=20&start=0")
  query_results = response.json()['documents']

  if not query_results: return None

  first_result = query_results[0]

  if first_result['gene_symbol'] == gene_symbol:
    return first_result['url'].split('/')[-1].strip()
  else:
    return None

In [43]:
# parsing test directory (old)
old_td_dict = collections.defaultdict(dict)

import re

for idx, row in old_td.iterrows():
  r_code_with_version = row['Test ID']
  ci = row['Clinical Indication']
  target = row['Target/Genes']

  if r_code_with_version.strip() in ['R14.1', 'R404.1', 'R404.3', 'R89.3', 'R246.1']: # edge case
      continue

  panel_number = re.search(r"\([^\]]*\)", target)
  if panel_number:
      number = panel_number.group(0)

      if '(' in number:
        number = number.split('(')[-1].strip()

      old_td_dict[r_code_with_version]['panelapp_id'] = number.lstrip('(').rstrip(')').strip()
  else:
    old_td_dict[r_code_with_version]['genes'] = []

    if ";" in target:
      for gene in target.split(';'):
        if ":" in gene:
          gene = gene.split(':')[-1].strip()
        elif 'and' in gene:
          for g in gene.split('and'):
            old_td_dict[r_code_with_version]['genes'].append(g.strip())
          continue

        else:
          gene = gene.strip()

        old_td_dict[r_code_with_version]['genes'].append(gene.strip())
      continue

    if "," in target:
      for gene in target.split(','):
        if 'and' in gene: #R221.1 in TD v5
          for g in gene.split('and'):
            old_td_dict[r_code_with_version]['genes'].append(g.strip())
        else:
          old_td_dict[r_code_with_version]['genes'].append(gene.strip())
      continue

    old_td_dict[r_code_with_version]['genes'].append(target.strip())

In [44]:
# parsing test directory (new)

new_td_dict = collections.defaultdict(dict)

import re

for idx, row in new_td.iterrows():
  r_code_with_version = row['Test ID']
  ci = row['Clinical Indication']
  target = row['Target/Genes']

  if r_code_with_version in ['R444.3', 'R14.1', 'R246.1', 'R89.3', 'R246.1', 'R404.1', 'R404.2', 'R404.3', 'R312.1', 'R312.2']: # edge case
    continue

  panel_number = re.search(r"\([^\]]*\)", target)
  if panel_number:
      number = panel_number.group(0)

      if '(' in number:
        number = number.split('(')[-1].strip()

      new_td_dict[r_code_with_version]['panelapp_id'] = number.lstrip('(').rstrip(')').strip()

  else:
    new_td_dict[r_code_with_version]['genes'] = []

    if ";" in target:
      for gene in target.split(';'):
        new_td_dict[r_code_with_version]['genes'].append(gene.strip())
      continue

    if "," in target:
      for gene in target.split(','):
        new_td_dict[r_code_with_version]['genes'].append(gene.strip())
      continue

    new_td_dict[r_code_with_version]['genes'].append(target.strip())

In [64]:
# the main block of code
from fuzzywuzzy import process

for r_code, v in old_r_code_to_panel.items():
  panel, panel_version = v.rsplit('_', maxsplit=1)

  if r_code in new_r_code_to_panel:
    print(f'{r_code} exist in new genepanel')

    # check panel version
    new_v = new_r_code_to_panel[r_code]
    new_panel, new_panel_version = new_v.rsplit('_', maxsplit=1)

    if v.strip() != new_v:
      print(f'!! Old panel {v} changed to new panel {new_v}')

    else:
      print('✅ Old and new panel is identical!')


    # check gene content between old and new genepanel
    old_genes = old_r_code_to_genes[r_code]
    new_genes = new_r_code_to_genes[r_code]

    if sorted(old_genes) == sorted(new_genes):
      print('✅ Similar gene content between old and new genepanel')
    else:
      print(f'❌ Gene content changed between old genepanel and new genepanel!')
      if set(new_genes) - set(old_genes):
        print(f'❌ These genes {set(new_genes) - set(old_genes)} have been added in new genepanel!')

      if set(old_genes) - set(new_genes):
        print(f'❌ These genes {set(old_genes) - set(new_genes)} have been removed in new genepanel!')

    # check old genepanel's gene content with panelapp
    panelapp_info = old_td_dict.get(r_code)

    if not panelapp_info:
      print(f"❌ R code {r_code} not present in old TD")
      print('\n')
      continue

    print(f'✅ R code found in old td')

    panelapp_id = panelapp_info.get('panelapp_id')

    if panelapp_id:
      panelapp_genes = get_genes_from_panelapp_id(panelapp_id, panel_version)

      if not panelapp_genes:
        print('🙈 Unable to get panels from PanelApp API\n')
        continue

      if panelapp_genes == set(old_genes):
        print(f'✅ old genepanel have similar gene content with Panelapp {panelapp_id} {panel_version}')
      else:
        if set(old_genes) - panelapp_genes:
          print(f'❌ {set(old_genes) - panelapp_genes} present in old genepanel but not in old PanelApp ({panelapp_id}) v{panel_version}')

        if panelapp_genes - set(old_genes): # if genes in panelapp but removed in old genepanel
          # check if RNAs thus the removal from genepanel
          removed_genes = panelapp_genes - set(old_genes)
          if removed_genes - rnas: # if removed genes are not in rnas
            print(f"❌ {removed_genes - rnas} in old genepanel and are not RNAs")
          else:
            print(f"✅ {len(removed_genes)} rnas gene removed in old genepanel but are present in PanelApp ({panelapp_id}) v{panel_version} which is correct!")

      # test gene content of new genepanel panel version
      if panel_version != new_panel_version:
        print(f'▲ Panel version has changed from v{panel_version} to v{new_panel_version} in new genepanel')
      else:
        print(f'Panel version remained unchanged v{panel_version} in new genepanel')

      new_panelapp_info = new_td_dict.get(r_code)

      if not new_panelapp_info:
        print(f"❌ R code {r_code} not present in new TD")
        print('\n')
        continue

      print(f'✅ R code found in new td')

      panelapp_id = new_panelapp_info.get('panelapp_id')

      if panelapp_id:
        panelapp_genes = get_genes_from_panelapp_id(panelapp_id, new_panel_version)

        if not panelapp_genes:
          print('🙈 Unable to get panels from PanelApp API\n')
          continue

        if panelapp_genes == set(new_genes):
          print(f'✅ new genepanel have similar gene content with Panelapp {panelapp_id} {new_panel_version}')
        else:
          if set(new_genes) - panelapp_genes:
            print(f'❌ {set(new_genes) - panelapp_genes} present in new genepanel but not in new PanelApp ({panelapp_id}) v{new_panel_version}')

          if panelapp_genes - set(new_genes):
            # check if RNAs thus the removal from genepanel
            removed_genes = panelapp_genes - set(new_genes)
            if removed_genes - rnas: # if removed genes are not in rnas
              print(f"❌ {removed_genes - rnas} in new genepanel and are not RNAs - should be included in genepanel?")
            else:
              print(f"✅ {len(removed_genes)} rnas gene removed in new genepanel but are present in PanelApp ({panelapp_id}) v{panel_version} which is correct!")

    else: # genes rather than panelapp id
      genes = panelapp_info.get('genes') # these are in gene format
      genes = [query_gene_symbol(g) for g in genes if query_gene_symbol(g)]

      # this should be compared with old test directory
      if set(old_genes) == set(genes):
        print('✅ similar gene content between old genepanel and old td')
      else:
        if set(genes) - set(old_genes):
            print(f'❌ {set(genes) - set(old_genes)} present in old td but not in old genepanel')

        if set(old_genes) - set(genes):
          print(f'❌ {set(old_genes) - set(genes)} present in old genepanel but not in old td')

      # comparing with new td and new genepanel (genes only)
      new_panelapp_info = new_td_dict.get(r_code)

      if not new_panelapp_info:
        print(f"❌ R code {r_code} not present in new TD")
        continue

      print(f'✅ R code found in new td!')

      panelapp_id = new_panelapp_info.get('panelapp_id')

      if panelapp_id:
        print(f'🙌 Panel now exist in new td as "{new_panel}" v{new_panel_version} rather than individual "HGNCs"')
        panelapp_genes = get_genes_from_panelapp_id(panelapp_id, new_panel_version)

        if not panelapp_genes:
          print('🙈 Unable to get panels from PanelApp API\n')
          continue

        if panelapp_genes == set(new_genes):
          print(f'✅ new genepanel have similar gene content with Panelapp {panelapp_id} {new_panel_version}')
        else:
          if set(new_genes) - panelapp_genes:
            print(f'❌ {set(new_genes) - panelapp_genes} present in old genepanel but not in new PanelApp ({panelapp_id}) v{new_panel_version}')

          if panelapp_genes - set(new_genes):
            print(f'❌ {panelapp_genes - set(new_genes)} present in PanelApp ({panelapp_id}) v{new_panel_version} but not in new genepanel')

      else:
        genes = new_panelapp_info.get('genes')
        genes = [query_gene_symbol(g) for g in genes if query_gene_symbol(g)]

        if set(new_genes) == set(genes):
          print('✅ similar gene content between new genepanel and new td')
        else:
          if set(genes) - set(new_genes):
              print(f'❌ {set(genes) - set(new_genes)} present in new td but not in new genepanel')

          if set(new_genes) - set(genes):
            print(f'❌ {set(new_genes) - set(genes)} present in new genepanel but not in new td')

  else:
    closest_r_match = process.extract(r_code, new_r_code_to_panel.keys(), limit=1)

    if closest_r_match[0][1] < 95:
      print(f'❌ There is no close match to r code in new genepanel: {r_code}, {panel}, {panel_version}')
      print(f'❌ This R code {r_code} is not found in new genepanel')

      if r_code not in old_td_dict:
        print(f'🔶 R code {r_code} not found in old TD\n')
      else:
        print(f'✅ R code found in old TD\n')

      if r_code not in new_td_dict:
        print(f'🔶 R code {r_code} not found in new TD\n')
      else:
        print(f'✅ R code found in new TD\n')
      continue
    else:
      print(f'🔵 Close match to {r_code} is {closest_r_match[0][0]} with match score {closest_r_match[0][1]}')
      # follow up action needed

  print('\n')

C1.1 exist in new genepanel
✅ Old and new panel is identical!
✅ Similar gene content between old and new genepanel
❌ R code C1.1 not present in old TD


C2.1 exist in new genepanel
✅ Old and new panel is identical!
✅ Similar gene content between old and new genepanel
❌ R code C2.1 not present in old TD


R100.3 exist in new genepanel
✅ Old and new panel is identical!
✅ Similar gene content between old and new genepanel
✅ R code found in old td
✅ old genepanel have similar gene content with Panelapp 168 4.0
Panel version remained unchanged v4.0 in new genepanel
✅ R code found in new td
✅ new genepanel have similar gene content with Panelapp 168 4.0


R101.1 exist in new genepanel
✅ Old and new panel is identical!
✅ Similar gene content between old and new genepanel
✅ R code found in old td
✅ old genepanel have similar gene content with Panelapp 53 3.0
Panel version remained unchanged v3.0 in new genepanel
✅ R code found in new td
✅ new genepanel have similar gene content with Panelapp 5

In [46]:
## new genepanel
print('checking new r code not in old genepanel...')
for r_code, v in new_r_code_to_panel.items():
  if r_code in old_r_code_to_panel:
    continue

  if r_code not in new_td_dict:
    print(f'❌ New R code in new genepanel: {r_code} not found in new TD')
    continue
  else:
    print(f'✅ R code {r_code} exist in new td')

  new_panel, new_panel_version = v.rsplit('_', maxsplit=1)

  new_panelapp_info = new_td_dict.get(r_code)
  panelapp_id = new_panelapp_info.get('panelapp_id')
  new_genes = new_r_code_to_genes[r_code]

  if panelapp_id:
    panelapp_genes = get_genes_from_panelapp_id(panelapp_id, new_panel_version)

    if not panelapp_genes:
      print('🙈 Unable to get panels from PanelApp API\n')
      continue

    if panelapp_genes == set(new_genes):
      print(f'✅ new genepanel have similar gene content with Panelapp {panelapp_id} {new_panel_version}')
    else:
      if set(new_genes) - panelapp_genes:
        print(f'❌ {set(new_genes) - panelapp_genes} present in old genepanel but not in new PanelApp ({panelapp_id}) v{new_panel_version}')

      if panelapp_genes - set(new_genes):
        # check if RNAs thus the removal from genepanel
        removed_genes = panelapp_genes - set(new_genes)
        if removed_genes - rnas: # if removed genes are not in rnas
          print(f"❌ {removed_genes - rnas} in new genepanel and are not RNAs - should be included in genepanel?")
        else:
          print(f"✅ {len(removed_genes)} rnas gene removed in new genepanel but are present in PanelApp ({panelapp_id}) v{panel_version} which is correct!")
        #print(f'❌ {panelapp_genes - set(new_genes)} present in PanelApp ({panelapp_id}) v{new_panel_version} but not in new genepanel')
  else:
    print(r_code, v)

  print('\n')

checking new r code not in old genepanel...
✅ R code R101.2 exist in new td
✅ new genepanel have similar gene content with Panelapp 53 3.0


✅ R code R102.2 exist in new td
✅ new genepanel have similar gene content with Panelapp 196 4.0


✅ R code R107.2 exist in new td
✅ new genepanel have similar gene content with Panelapp 543 2.0


✅ R code R110.2 exist in new td
✅ new genepanel have similar gene content with Panelapp 98 3.3


✅ R code R124.2 exist in new td
✅ new genepanel have similar gene content with Panelapp 517 1.2


✅ R code R127.2 exist in new td
✅ new genepanel have similar gene content with Panelapp 76 3.1


✅ R code R128.2 exist in new td
✅ new genepanel have similar gene content with Panelapp 13 3.2


✅ R code R129.2 exist in new td
✅ new genepanel have similar gene content with Panelapp 214 4.0


✅ R code R130.2 exist in new td
✅ new genepanel have similar gene content with Panelapp 224 3.1


✅ R code R131.2 exist in new td
✅ new genepanel have similar gene content with